# Evolutionary Algorithm fit of network

In [1]:
# whenever changes are made to any imported files this will reload them automatically
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow_probability as tfp

print(tf.__version__)

2.2.0


In [439]:
from BHDVCStf import BHDVCS #modified bhdvcs file

In [3]:
class DvcsData(object):
    def __init__(self, df):
        self.X = df.loc[:, ['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'ReH', 'ReE', 'ReHtilde', 'dvcs']]
        self.XnoCFF = df.loc[:, ['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'dvcs']]
        self.y = df.loc[:, 'F']
        self.Kinematics = df.loc[:, ['k', 'QQ', 'x_b', 't']]
        self.erry = df.loc[:, 'errF']
        self.for_bhdvcs = (df.loc[:, 'phi_x'], df.loc[:, 'k'], df.loc[:, 'QQ'], df.loc[:, 'x_b'], df.loc[:, 't'],
         df.loc[:, 'F1'], df.loc[:, 'F2'], df.loc[:, 'dvcs'])
        
    def __len__(self):
        return len(self.X)
    
    def getSet(self, setNum, itemsInSet=36):
        pd.options.mode.chained_assignment = None
        subX = self.X.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1, :]
        subX['F'] = self.y.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1]
        subX['errF'] = self.erry.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1]
        pd.options.mode.chained_assignment = 'warn'
        return DvcsData(subX)
    
    def sampleY(self):
        return np.random.normal(self.y, self.erry)
    
    def sampleWeights(self):
        return 1/self.erry
    
    def for_bhdvcs(self):
        return 

In [4]:
df = pd.read_csv('dvcs_xs_newsets_genCFFs.csv')

In [5]:
data = DvcsData(df)
set0 = data.getSet(0)

In [317]:
df

,#Set,index,k,QQ,x_b,t,phi_x,F,errF,F1,F2,dvcs,ReH,ReE,ReHtilde
0,0,0,3.75,1.74013,0.435095,-0.380868,0,0.058205,0.002910,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
1,0,1,3.75,1.74013,0.435095,-0.380868,10,0.055596,0.002780,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
2,0,2,3.75,1.74013,0.435095,-0.380868,20,0.066353,0.003318,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
3,0,3,3.75,1.74013,0.435095,-0.380868,30,0.068655,0.003433,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
4,0,4,3.75,1.74013,0.435095,-0.380868,40,0.072765,0.003638,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,14,31,7.75,2.63258,0.345012,-0.361188,310,0.023981,0.001199,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283
536,14,32,7.75,2.63258,0.345012,-0.361188,320,0.025345,0.001267,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283
537,14,33,7.75,2.63258,0.345012,-0.361188,330,0.024191,0.001210,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283
538,14,34,7.75,2.63258,0.345012,-0.361188,340,0.020530,0.001027,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283


## define genetic algorithm function

In [443]:
class TotalUUXS(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TotalUUXS, self).__init__(**kwargs)
        self.F = BHDVCS()
        
    def call(self, inputs):
        return self.F.TotalUUXS(inputs[:, :8], inputs[:, 8], inputs[:, 9], inputs[:, 10])
    
    def get_config(self):
        config = super(TotalUUXS, self).get_config()
        return config

In [444]:
kinematics = tf.keras.Input(shape=(4))
x = tf.keras.layers.Dense(20, activation="tanh")(kinematics)
outputs = tf.keras.layers.Dense(3)(x) #three output nodes for ReH, ReE, ReHtilde
noncffInputs = tf.keras.Input(shape=(8))
totalUUXSInputs = tf.keras.layers.concatenate([noncffInputs, outputs])
F = TotalUUXS(dtype='float32')(totalUUXSInputs)

globalModel = tf.keras.Model(inputs=[kinematics, noncffInputs], outputs=F)

In [445]:
custom_objects = {"TotalUUXS": TotalUUXS}
with tf.keras.utils.custom_object_scope(custom_objects):
    t = tf.keras.models.clone_model(globalModel)

In [461]:
models = []
population = []
for i in range(4):
    with tf.keras.utils.custom_object_scope(custom_objects):
        models.append(tf.keras.models.clone_model(globalModel))
    population.append(models[i].get_weights())

In [447]:
def pad(member):
    member = [member[0], np.expand_dims(member[1], 1), member[2], np.expand_dims(member[3], 1)]
    max_d1 = -1
    max_d2 = -1
    for layer in member:
        if layer.shape[0] > max_d1:
            max_d1 = layer.shape[0]
        if layer.shape[1] > max_d2:
            max_d2 = layer.shape[1]
    rect = []
    for layer in member:
        to_pad_d1 = max_d1 - layer.shape[0]
        to_pad_d2 = max_d2 - layer.shape[1]
        rect.append(np.pad(layer, ((0, to_pad_d1), (0, to_pad_d2))))
    return tf.convert_to_tensor(rect)

In [448]:
def unpad(member):
    return [member[0][:4, :], member[1][:, 0], member[2][:, :3], member[3][:3, 0]]

In [462]:
population = list(map(pad, population))

In [450]:
def loss(model, inputs, targets, sample_weight):
    error = model(inputs) - targets
    return tf.reduce_mean(tf.square(error) * sample_weight)

In [451]:
def objective(*weights):
    losses = []
    for i in range(len(weights)):
        model = models[i]
        model.set_weights(unpad(weights[i]))
        losses.append(loss(model, [set0.Kinematics, set0.XnoCFF], set0.y, set0.sampleWeights()**2))
    print(tf.convert_to_tensor(losses))
    return tf.convert_to_tensor(losses)

In [245]:
globalModel.compile(
    optimizer = tf.keras.optimizers.Adam(1),
    loss = tf.keras.losses.MeanSquaredError(),
)

In [452]:
res = tfp.optimizer.differential_evolution_minimize(objective, population,max_iterations=2000, func_tolerance=10.)

tf.Tensor([164.48836  45.88693 161.6081   86.50659], shape=(4,), dtype=float32)
tf.Tensor([260.7067   93.26495 183.70557 328.22498], shape=(4,), dtype=float32)
tf.Tensor([233.84193 271.3201  203.75159 209.89178], shape=(4,), dtype=float32)
tf.Tensor([264.69037  178.17491   91.971504 345.46597 ], shape=(4,), dtype=float32)
tf.Tensor([185.10509 213.93428 242.58675 160.5495 ], shape=(4,), dtype=float32)
tf.Tensor([125.42735 219.35355 268.95233 155.5299 ], shape=(4,), dtype=float32)
tf.Tensor([266.0715  272.8297  282.23697 279.18924], shape=(4,), dtype=float32)
tf.Tensor([253.9783  273.26645 264.60287 242.38167], shape=(4,), dtype=float32)
tf.Tensor([195.48462 177.3103  268.29556 211.10023], shape=(4,), dtype=float32)
tf.Tensor([279.04227 283.39838 241.09003 271.1476 ], shape=(4,), dtype=float32)
tf.Tensor([231.91612 215.77185 209.98195 239.95749], shape=(4,), dtype=float32)
tf.Tensor([250.50226 273.41296 280.69785 232.7276 ], shape=(4,), dtype=float32)
tf.Tensor([288.5887  291.48468 270.3

tf.Tensor([231.86642 211.54951 288.27866 225.39378], shape=(4,), dtype=float32)
tf.Tensor([246.53055 243.64972 284.94598 241.15472], shape=(4,), dtype=float32)
tf.Tensor([246.31665 186.67622 249.3118  258.71262], shape=(4,), dtype=float32)


KeyboardInterrupt: 

In [254]:
best = res[2]

In [256]:
globalModel.set_weights(unpad(best))

In [257]:
def cffs_from_globalModel(model):
    subModel = tf.keras.backend.function(model.layers[0].input, model.layers[3].output)
    return subModel(np.asarray(set0.Kinematics)[None, 0])[0]

In [258]:
cffs_from_globalModel(globalModel)

array([ 0.29813153, -0.01019754,  0.92896974], dtype=float32)

# Direct fit of compton form factors with differential evolution

In [6]:
from ROOT import gROOT

gROOT.LoadMacro('F.C')
from ROOT import F
f = F()

Welcome to JupyROOT 6.22/00


In [7]:
def TotalUUXS_curve_fit(fixed, ReH, ReE, ReHtilde):
    phi, k, QQ, xB, t, F1, F2, dvcs = tuple(map(list, fixed))
    fs = np.zeros(len(phi))
    for i in range(len(fs)):
        fs[i] = f.TotalUUXS(phi[i], k[i], QQ[i], xB[i], t[i], F1[i], F2[i], ReH, ReE, ReHtilde, dvcs[i])
    return fs

In [11]:
df = pd.read_csv("dvcs_xs_newsets_genCFFs.csv")
data = DvcsData(df)
setI = data.getSet(0)

In [12]:
population = [tf.convert_to_tensor([np.random.uniform(-100, 100) for i in range(3)]) for i in range(8)]

In [13]:
population

[<tf.Tensor: shape=(3,), dtype=float32, numpy=array([28.75297 , 23.30636 , 46.000076], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 63.242134, -15.610227, -79.790375], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-20.857727,  43.334515, -64.81432 ], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 94.02247, -99.63276, -95.81258], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-4.1308866, 82.913185 , 92.122856 ], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 51.566856, -89.943054, -54.267056], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 56.383347, -88.86382 ,  89.83965 ], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([  1.7033573,  19.28127  , -55.664505 ], dtype=float32)>]

In [14]:
setI = data.getSet(0)

In [63]:
def objective(*cffs):
    losses = []
    cffs = cffs[0]
    for cff in cffs:
        losses.append(np.mean(np.square((TotalUUXS_curve_fit(setI.for_bhdvcs, cff[0], cff[1], cff[2]) - setI.y))
                   * np.square(setI.sampleWeights())))
    return tf.convert_to_tensor(losses)

In [64]:
population_size = 10
population = (tf.random.uniform([population_size, 3], -100, 100),)

In [134]:
res = tfp.optimizer.differential_evolution_minimize(objective, initial_population = population,
                                                    max_iterations=500, crossover_prob = .8)

In [135]:
res.position

[<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 11.950319, -48.957527,   7.393045], dtype=float32)>]

In [136]:
res.objective_value

<tf.Tensor: shape=(), dtype=float64, numpy=0.8751671681347467>

## Evaluate

In [137]:
numSamples = 20
population_size = 10

for i in tqdm(range(max(df['#Set'])+1)):
    setI = data.getSet(i)
    
    results = pd.DataFrame({
      "ReH": np.zeros(numSamples),
      "ReE": np.zeros(numSamples),
      "ReHtilde": np.zeros(numSamples)
    })
    
    losses = 0
    
    for sample in range(numSamples):
        
        population = (tf.random.uniform([population_size, 3], -100, 100),)
        
        res = tfp.optimizer.differential_evolution_minimize(objective, initial_population=population,
                                                            max_iterations=2000)
        
        for num, cff in enumerate(['ReH', 'ReE', 'ReHtilde']):
            results.loc[sample, cff] = float(res.position[0][num])
        
        losses += res.objective_value
        
    print("\nSet " + str(i) + ":")
    print("Avg Loss: " + str(losses/numSamples))
    print(pd.DataFrame({'Fitted': list(results.mean()),
                'Error': list(results.std()),
                'True': list(df.loc[36*i, ['ReH', 'ReE', 'ReHtilde']])},
             ['ReH', 'ReE', 'ReHTilde']))

  7%|▋         | 1/15 [14:23<3:21:35, 864.00s/it]


Set 0:
Avg Loss: tf.Tensor(1.4496975246772856, shape=(), dtype=float64)
             Fitted      Error      True
ReH       12.746454   4.791278  13.05540
ReE      -51.592255  24.698525 -53.05540
ReHTilde   6.741661   0.612975   7.25302


 13%|█▎        | 2/15 [29:03<3:08:12, 868.62s/it]


Set 1:
Avg Loss: tf.Tensor(1.0778639924088909, shape=(), dtype=float64)
             Fitted      Error      True
ReH        8.929993   6.048974  12.55490
ReE      -33.039746  31.899378 -52.55490
ReHTilde   8.882531   3.081388   6.97494


 20%|██        | 3/15 [43:35<2:53:57, 869.81s/it]

...


Set 12:
Avg Loss: tf.Tensor(1.499385197309883, shape=(), dtype=float64)
             Fitted      Error      True
ReH        5.786214   5.427584   6.99014
ReE      -39.351003  33.485238 -46.99010
ReHTilde   2.420396   2.130612   3.88341


 93%|█████████▎| 14/15 [3:35:10<16:27, 987.41s/it] 


Set 13:
Avg Loss: tf.Tensor(3.4335167921138363, shape=(), dtype=float64)
             Fitted      Error      True
ReH        3.638039  10.130620   7.22424
ReE      -26.659518  57.991056 -47.22420
ReHTilde   1.478217   9.900955   4.01347


100%|██████████| 15/15 [3:51:09<00:00, 924.64s/it]


Set 14:
Avg Loss: tf.Tensor(3.2136060787792706, shape=(), dtype=float64)
            Fitted      Error      True
ReH       3.205575  13.396316  11.74110
ReE      -4.237342  74.654976 -51.74110
ReHTilde  6.680806   1.244338   6.52283
